# Jupyter Example
## LAMMPS: body/in.squares
Reference: https://github.com/lammps/lammps/blob/master/examples/body/in.squares  
Required files: data.squares  
Description: 2d rounded polygon bodies

In [1]:
from __future__ import print_function
import random, math
from lammps import IPyLammps

In [2]:
L = IPyLammps()

L.units("lj")
L.dimension("2")
L.atom_style("body rounded/polygon 1 6")
L.atom_modify("map array")
L.read_data("data.squares")

LAMMPS output is captured by PyLammps wrapper


['Reading data file ...',
 '  orthogonal box = (0 0 -0.5) to (12 12 0.5)',
 '  1 by 1 by 1 MPI processor grid',
 '  reading atoms ...',
 '  2 atoms',
 '  2 bodies']

In [3]:
L.variable("r index 4")
L.variable("steps index 10000")
L.variable("T index 0.5")
L.variable("P index 0.1")
L.variable("seed index 980411")
L.replicate("$r $r 1")
L.velocity("all create $T ${seed} dist gaussian mom yes rot yes")

In [4]:
L.variable("cut_inner equal 0.5")
L.variable("k_n equal 100")
L.variable("k_na equal 2")
L.variable("c_n equal 1")
L.variable("c_t equal 1")
L.variable("mu equal 0.1")
L.variable("delta_ua equal 0.5")
L.pair_style("body/rounded/polygon ${c_n} ${c_t} ${mu} ${delta_ua} ${cut_inner}")
L.pair_coeff("* * ${k_n} ${k_na}")
L.comm_modify("vel yes")

In [5]:
L.neighbor("0.5 bin")
L.neigh_modify("every 1 delay 0 check yes")

#L.fix("1 all nve/body")
#L.fix("1 all nvt/body temp $T $T 1.0")
L.fix("1 all npt/body temp $T $T 1.0 x 0.001 $P 1.0 y 0.001 $P 1.0 couple xy fixedpoint 0 0 0")
L.fix("2 all enforce2d")

In [6]:
L.compute("1 all body/local id 1 2 3")
L.dump("1 all local 1000 dump.polygon.* index c_1[1] c_1[2] c_1[3] c_1[4]")

#L.dump("2 all image 1000 image.*.jpg type type zoom 2.0 adiam 1.5 body type 0 0")
#L.dump_modify("2 pad 6")

L.dump("2 all movie 1000 mov.mp4 type type zoom 1.0 adiam 1.5 body type 0 0")
L.dump_modify("2 pad 6")

In [7]:
L.thermo("1000")
L.thermo_style("custom step ke pe etotal press")
L.timestep("0.001")
L.run("${steps}")

['Neighbor list info ...',
 '  update every 1 steps, delay 0 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 6.15685',
 '  ghost atom cutoff = 6.15685',
 '  binsize = 3.07843, bins = 16 16 1',
 '  1 neighbor lists, perpetual/occasional/extra = 1 0 0',
 '  (1) pair body/rounded/polygon, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/atomonly/newton',
 '      stencil: half/bin/2d/newton',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : lj',
 '  Current step  : 0',
 '  Time step     : 0.001',
 'Per MPI rank memory allocation (min/avg/max) = 6.342 | 6.342 | 6.342 Mbytes',
 'Step KinEng PotEng TotEng Press ',
 '       0     0.484375         0.25     0.734375 0.0067274306 ',
 '    1000   0.47546844 0.0017172058   0.47718564 0.0035093065 ',
 '    2000   0.63963318   0.20530714   0.84494032  0.023913933 ',
 '    3000   0.81423506  0.091172659   0.90540772 -0.0028257245 ',
 '    4000

In [8]:
L.undump(2)
L.video("mov.mp4")